# Use the `DockerImage` class to dockerize a python function

In [1]:
import boto3
import logging
import uuid

# This example is housed in cloudknot/examples
# With cloudknot installed, you may simply import with
# `import cloudknot as ck`
# However, if you havent installed cloudknot yet
# (e.g. you're running this example from a fresh clone of the cloudknot github repo),
# you should temporarily add cloudknot to your sys.path using the commands below
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import cloudknot as ck

Set the logging level to print loging statements in the notebook

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Change this to logging.DEBUG if you want more verbose output

## Initialization

Create a test function, `test_func`, that `DockerImage` will dockerize. `DockerImage` uses [`clize`](https://clize.readthedocs.io/en/stable/) to convert the input function argument into a command line interface. By default, it will treat all of your parameters as positional parameters. But you can change this behavior, e.g. by  [accepting options](https://clize.readthedocs.io/en/stable/basics.html#accepting-options). Read the `clize` docs for more detail.

In [3]:
def test_func(name=None, *, no_capitalize=False):
    """Test function for unit testing of cloudknot.DockerReqs

    Import statements of various formats are deliberately scattered
    throughout the function to test the pipreqs components of
    clouknot.DockerReqs
    """
    import AFQ
    import sys
    import boto3.ec2
    if name:
        from docker import api
        from os.path import join

        if not no_capitalize:
            import pytest as pt
            import h5py.utils as h5utils

            name = name.title()

        return 'Hello {0}!'.format(name)

    from six import binary_type as bt
    from dask.base import curry as dbc

    return 'Hello world!'

Create a `DockerImage` instance from the test function. Since `AFQ` is not available via `pip install`, we should get a warning. Print some of the `DockerImage` parameters.

In [4]:
image = ck.DockerImage(func=test_func)
print(image.name)
print(image.build_path)
print(image.req_path)
print(image.docker_path)
print(image.script_path)
print(image.pip_imports)
print(image.missing_imports)
print(image.username)
print(image.func)

INFO:root:Wrote python function test_func to script /Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_464zlpq8/test_func.py
DEBUG:root:Found packages: {'h5py', 'AFQ', 'dask', 'docker', 'os', 'boto3', 'pytest', 'six', 'sys', 'clize'}
DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): pypi.python.org
DEBUG:requests.packages.urllib3.connectionpool:https://pypi.python.org:443 "GET /pypi/AFQ/json HTTP/1.1" 404 25
DEBUG:root:Package AFQ does not exist or network problems
DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): pypi.python.org
DEBUG:requests.packages.urllib3.connectionpool:https://pypi.python.org:443 "GET /pypi/boto3/json HTTP/1.1" 200 72641
DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): pypi.python.org
DEBUG:requests.packages.urllib3.connectionpool:https://pypi.python.org:443 "GET /pypi/clize/json HTTP/1.1" 200 25963
DEBUG:requests.packages.urllib3.connectionpool:

test_func
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_464zlpq8
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_464zlpq8/requirements.txt
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_464zlpq8/Dockerfile
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_464zlpq8/test_func.py
[{'name': 'boto3', 'version': '1.4.7'}, {'name': 'clize', 'version': '4.0.1'}, {'name': 'dask', 'version': '0.15.4'}, {'name': 'docker', 'version': '2.5.1'}, {'name': 'h5py', 'version': '2.7.1'}, {'name': 'pytest', 'version': '3.2.3'}, {'name': 'six', 'version': '1.11.0'}]
['AFQ']
cloudknot-user
<function test_func at 0x10ad78e18>


## Building the image

On `__init__`, `DockerImage` just creates all of the pre-requisite files: a Dockerfile, requirements.txt, and a CLI script. Since we haven't built or pushed this docker image yet, some of the parameters are `None`.

In [5]:
print(image.images)
print(image.repo_uri)

[]
None


Let's actually build this image locally. You'll need Docker running on your machine for this to work. We'll pass in a list of tags. You can use any tag you want, as long as it isn't "latest."

In [6]:
image.build(tags=['example'])

INFO:root:Using cloudknot config file /Users/Adam/.aws/cloudknot
DEBUG:docker.auth:Trying paths: ['/Users/Adam/.docker/config.json', '/Users/Adam/.dockercfg']
DEBUG:docker.auth:Found file at path: /Users/Adam/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'HttpHeaders' section
DEBUG:docker.auth:Found 'credsStore' section
INFO:root:Building image cloudknot/test_func with tag example
DEBUG:docker.api.build:Looking for auth config
DEBUG:docker.auth:Using credentials store

Now that we've built the image, the `image_name` and `tags` parameters should be set. Since we haven't pushed the image to a repository, the `repo_uri` parameter is still `None`.

In [7]:
print(image.images)
print(image.repo_uri)

[{'name': 'cloudknot/test_func', 'tag': 'example'}]
None


## Pushing to a remote repository (by specifying a `DockerRepo` instance)

`DockerImage` instances know how to interact with `DockerRepo` instances. So instead of creating the repository using boto3 (as above), we could instead have created a new one using cloudknot's `DockerRepo` class.

In [8]:
repo = ck.aws.DockerRepo(name=str(uuid.uuid4()))
image.push(repo=repo)

DEBUG:botocore.hooks:Event before-parameter-build.ecr.DescribeRepositories: calling handler <function generate_idempotent_uuid at 0x10ea42d08>
DEBUG:botocore.endpoint:Making request for OperationModel(name=DescribeRepositories) (verify_ssl=True) with params: {'context': {'has_streaming_input': False, 'client_config': <botocore.config.Config object at 0x10fa56c88>, 'auth_type': None, 'client_region': 'us-east-1'}, 'body': b'{"repositoryNames": ["7c8959d5-593b-4efa-9080-2356407ab96e"]}', 'url': 'https://ecr.us-east-1.amazonaws.com/', 'headers': {'X-Amz-Target': 'AmazonEC2ContainerRegistry_V20150921.DescribeRepositories', 'User-Agent': 'Boto3/1.4.2 Python/3.5.3 Darwin/16.7.0 Botocore/1.6.3', 'Content-Type': 'application/x-amz-json-1.1'}, 'url_path': '/', 'query_string': '', 'method': 'POST'}
DEBUG:botocore.hooks:Event request-created.ecr.DescribeRepositories: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x10fa56be0>>
DEBUG:botocore.hooks

Get the URI of the remote repo

In [9]:
print(image.repo_uri)

455598791984.dkr.ecr.us-east-1.amazonaws.com/7c8959d5-593b-4efa-9080-2356407ab96e


Clean up after ourselves using the `clobber` method. This will remove the associated local docker images and the pre-requisite files. By design, it will not delete the remote repository or remote image. This must be done by calling the `DockerRepo` `clobber` method.

In [10]:
image.clobber()
repo.clobber()

INFO:root:Removed /Users/Adam/code/projects/cloudknot/cloudknot_docker_test_func_4kapkrhv/test_func.py
INFO:root:Removed /Users/Adam/code/projects/cloudknot/cloudknot_docker_test_func_4kapkrhv/Dockerfile
INFO:root:Removed /Users/Adam/code/projects/cloudknot/cloudknot_docker_test_func_4kapkrhv/requirements.txt
INFO:root:Removed /Users/Adam/code/projects/cloudknot/cloudknot_docker_test_func_4kapkrhv
DEBUG:docker.auth:Trying paths: ['/Users/Adam/.docker/config.json', '/Users/Adam/.dockercfg']
DEBUG:docker.auth:Found file at path: /Users/Adam/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for {0} is absent. Client might be using a credentials store instead.
DEB